In [ ]:
import sqlite3

import pandas as pd
import pymysql

In [ ]:
conn = sqlite3.connect('sakila.db')
film_df = pd.read_sql("select * from film;", conn)
film_actor_df = pd.read_sql("select * from film_actor;", conn)
actor_df = pd.read_sql("select * from actor;", conn)


# desc

In [ ]:
"""desc film"""
film_df.info()

In [ ]:
"""
select * 
from film 
where 1 = 0
"""
film_df.columns.values

In [ ]:
film_df.shape

# select 

In [ ]:
"""
select * 
from film;
"""
film_df

In [ ]:
"""
select * 
from film 
limit 3;
"""
film_df.head(3) # we can also transepose the result by add .T, film_df.head(3).T

In [ ]:
"""
select film_id, title, description
from film
limit 10, 5;
"""
film_df[["film_id", "title", "description"]][10:10 + 5] # in sql 10 is offset, 5 is limit, so [10:10 + 5] in pandas

In [ ]:
"""
select distinct rating
from film;
"""
film_df['rating'].unique()

# count

In [ ]:
"""
select count(*)
from film;
"""

len(film_df)

In [ ]:
"""
select count(distinct rating)
from film;
"""

len(film_df['rating'].unique())

In [ ]:
"""
"""

# where

In [ ]:
"""
select film_id, title, description
from film
where film_id = 10;
"""

film_df[film_df["film_id"] == 10][["film_id", "title", "description"]]

In [ ]:
"""
select film_id, title, description
from film
where rental_rate > 2
      and length < 120;
"""

film_df[(film_df["rental_rate"] > 2) & (film_df["length"] < 120)][["film_id", "title", "description"]]

# in

In [ ]:
"""
SELECT  * 
FROM sakila.film
where rental_duration in (3,6,7);
"""

film_df[film_df.rental_duration.isin([3,6,7])]

In [ ]:
"""
SELECT  * 
FROM sakila.film
where rental_duration not in (3,6,7);
"""

film_df[~film_df.rental_duration.isin([3,6,7])].head()

# NULL

In [ ]:
"""
SELECT * 
FROM sakila.film
where original_language_id is null;
"""
film_df[film_df.original_language_id.isnull()].head()

# fill NULL
补全NULL值的操作pandas比sql方便

In [ ]:
"""
SELECT (case when original_language_id is null then 999 else original_language_id end ) as original_language_id
FROM sakila.film
where original_language_id is null;
"""

film_df.original_language_id.fillna(999,inplace=True)
film_df.head()

# order by

In [ ]:
"""
select rental_rate, film_id, title, description
from film
order by rental_rate
"""
 
film_df.sort_values(['rental_rate'])[["rental_rate", "film_id", "title", "description"]]  # default asc, not inplace

In [ ]:
"""
select rental_rate, film_id, title, description
from film
order by rental_rate desc
"""
 
film_df.sort_values(['rental_rate'], ascending=0)[["rental_rate", "film_id", "title", "description"]]  # desc, not inplace

# group by

In [ ]:
"""
select rating, count(*)
from film
group by rating
"""

film_df.rating.value_counts()

In [ ]:
"""
select release_year, rating, count(*)
from film
group by release_year, rating
"""

film_df.groupby(["release_year", "rating"]).size()  # the result type is series, we can trans it to Dataframe by reset_index

In [ ]:
"""
select release_year, rating, count(*) as counts
from film
group by release_year, rating
"""

film_df.groupby(["release_year", "rating"]).size().reset_index(name="counts")  # the result type is Dataframe and rename the column

In [ ]:
"""
select  rating , count(distinct rental_duration)  as rental_duration_type_count
from  film
group by rating;
"""

film_df.groupby('rating').rental_duration.nunique().reset_index(name="rental_duration_type_count")

In [ ]:
"""
select * 
from film
where special_features in (
	select special_features
	from film
	group by special_features
	having COUNT(*) > 78)
"""

film_df.groupby('special_features').filter(lambda x: len(x) > 78)

In [ ]:
"""
select rating, count(distinct length) as length_distinct_count, avg(length) as length_mean, avg(rental_rate) as rental_rate_mean
from film
group by rating;
"""

## [DEPRECATED] Dictionary groupby format
使用一种叫Dictionary groupby format的方式，然后droplevel(0)，最后reset_index

In [ ]:
new_df = film_df.groupby("rating").agg({"length": {"length_distinct_count": lambda x: x.nunique(), 
                                                   "length_mean": "mean"},
                                        "rental_rate": {"rental_rate_mean": "mean"}})
new_df

In [ ]:
new_df.columns = new_df.columns.droplevel(0)
new_df

In [ ]:
new_df.reset_index()

## Use 'named' functions instead of lambda's:
但是上面用嵌套字典的来重命名计算的字段的方式已经[DEPRECATED]了 (>=0.20.1),推荐的方式是用函数代替匿名函数

In [ ]:
def length_distinct_count(group):
    return group.nunique()
    
def length_mean(group):
    return group.mean()
    
def rental_rate_mean(group):
    return group.mean()
    
new_df = film_df.groupby("rating").agg({"length": [length_distinct_count, length_mean],
                                        "rental_rate": rental_rate_mean})
new_df

# 后面的做法同上

# group_concat
oralce: wm_concat*

In [ ]:
"""
SELECT actor_id, group_concat(film_id order by film_id separator ',')  as film_ids
FROM sakila.film_actor
group by actor_id;
"""

film_actor_df['film_id_str'] = film_actor_df['film_id'].map(str)
film_actor_df.groupby('actor_id')['film_id_str'].apply(lambda x: ','.join(x)).reset_index(name="film_ids").head()